# Deploying a Dask Hub

All this material is taken from the following docs:
- https://docs.dask.org/en/latest/setup/kubernetes-helm.html
- https://zero-to-jupyterhub.readthedocs.io/en/latest/kubernetes/setup-kubernetes.html
- https://zero-to-jupyterhub.readthedocs.io/en/latest/kubernetes/setup-helm.html

## Creating a Kubernetes Cluster

First, you need to enable the Kubernetes API if not already done:
1. Go to console.cloud.google.com
2. Select the Kubernetes Engine in the menu: https://console.cloud.google.com/marketplace/product/google/container.googleapis.com
3. Enable the API.

Then you'll need a terminal with __gcloud__ and __kubectl__. The simplest is just to use the Google Cloud Shell from console.cloud.google.com. If you prefer, you can follow the links below to find how to install everything on your computer.

Ask Google Cloud to create a managed Kubernetes cluster and a default node pool to get nodes from:

```
gcloud container clusters create \
  --machine-type n1-standard-4 \
  --enable-autoscaling \
  --min-nodes 1 \
  --max-nodes 10 \
  --num-nodes 1 \
  --zone europe-west1-b \
  --cluster-version latest \
  dask-hub-k8s
```

You can test if the cluster is running:
```
kubectl get node
```

Then get permissions to perform all administrative actions needed. Don't forget to replace your email below.

```
kubectl create clusterrolebinding cluster-admin-binding \
  --clusterrole=cluster-admin \
  --user=<GOOGLE-EMAIL-ACCOUNT>
```

## Setting up Helm

From your Google Cloud Shell or terminal:

```
curl https://raw.githubusercontent.com/helm/helm/master/scripts/get-helm-3 | bash
helm list
```

should return:
```
NAME    NAMESPACE       REVISION        UPDATED STATUS  CHART   APP VERSION
```


## Helm install a Dask Hub for multiple users

We install a full Dask up for multiple users because:
- It enables automatic scaling of computing resources,
- If some of you is unable to deploy its Dask cluster, she or he will be able to use the cluster of someone else.

Verify that you’ve set up a Kubernetes cluster and added Dask’s helm charts:

```
helm repo add dask https://helm.dask.org/
helm repo update
```

Generate tokens to configure Jupyterhub and Dask-gateway:

```
openssl rand -hex 32  # generate token-1
openssl rand -hex 32  # generate token-2
```

Create the file below (for example using vi or any editor) and substitute those two values for `<token-1>` and `<token-2>`.
    
```
# file: secrets.yaml
jupyterhub:
  proxy:
    secretToken: "<token-1>"
  hub:
    services:
      dask-gateway:
        apiToken: "<token-2>"
  scheduling:
    podPriority:
      enabled: true
    userPlaceholder:
      # Specify three dummy user pods will be used as placeholders
      replicas: 3
    userScheduler:
      enabled: true

dask-gateway:
  gateway:
    auth:
      jupyterhub:
        apiToken: "<token-2>"
    extraConfig:
      optionHandler: |
        from dask_gateway_server.options import Options, Integer, Float

        def options_handler(options):
          return {
            "worker_cores": options.worker_cores,
            "worker_memory": int(options.worker_memory * 2 ** 30),
          }

        c.Backend.cluster_options = Options(
          Integer("worker_cores", default=1, min=1, max=4, label="Worker Cores"),
          Float("worker_memory", default=1, min=1, max=8, label="Worker Memory (GiB)"),
          handler=options_handler,
        )
```

Now we just install Dask Hub:
```
helm upgrade --wait --install --render-subchart-notes \
    --namespace daskhub \
    --create-namespace \
    dhub dask/daskhub \
    --values=secrets.yaml
```

## Check install and go to Jupyter!

To get the public IP of your hub deployment:
```
kubectl --namespace=daskhub get service proxy-public
```

Get the external IP, and open it in your browser. You should be able to login with any username/password